In [3]:
import pandas as pd
import numpy as np

In [4]:
impressions_df = pd.read_csv("../datasets/impressions/impressions_groupm_tele2_6_months.csv")

In [5]:
omg_impressions = pd.read_excel("../datasets/Xandr_data_attention.xlsx")
omg_impressions

,Placement Id,Placement Name,Size,Site Domain,Imps
0,19499407,SE-Aftonbladet-wde-Article-Insider_1,300x250,aftonbladet.se,277434
1,19499466,SE-Aftonbladet-wph-Article-Panorama_1,320x320,aftonbladet.se,231478
2,19499490,SE-Aftonbladet-wph-Front-Toppanorama,320x320,aftonbladet.se,219650
3,20148636,Insider,300x250,expressen.se,206005
4,20148697,Insider,300x250,dn.se,204883
...,...,...,...,...,...
586,22875680,Outsider,300x600,tidningenharjedalen.se,1
587,19499407,SE-Aftonbladet-wde-Article-Insider_1,300x250,purring-panther.sa.com,1
588,19499410,SE-Aftonbladet-wde-Article-Modul_1,640x320,aftonbladet.se,1
589,19499407,SE-Aftonbladet-wde-Article-Insider_1,300x250,convox.cloud,1


In [6]:
# run through predictions
import os
import json


def _get_pred_map():
    pred_map = dict()
    for _i in range(1,2):
        if _i == 1:
            _year = 2022
            _range1 = 11
            _range2 = 13
        else:
            _year = 2023
            _range1 = 1
            _range2 = 5
        for i in range(_range1, _range2):
            if i < 10:
                i = f'0{i}'
            for day in os.listdir(f'../predictions/{_year}/{i}'):
                for h in os.listdir(f'../predictions/{_year}/{i}/{day}'):
                    if not h.endswith('.ndjson'):
                        continue
                    with open(f'../predictions/{_year}/{i}/{day}/{h}')as f:
                        for line in f.readlines():
                            json_line = json.loads(line)
                            id_ = json_line['id']
                            pred_map[id_] = json_line['prediction']
    return pred_map
                            
pred_map = _get_pred_map()

In [7]:
import json

def extract_pid(placement_id):
    placement_id = json.loads(placement_id)
    bam_ad_slots = placement_id["bam_ad_slot"]
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    ret = ''
    try:
        ret = placement_id['tag_id'][0]
    except:
        ret = ','.join([bas for bas in final_bam_ad_slots])
    else:
        ret += ','.join([bas for bas in final_bam_ad_slots])
    return ret
    

def pid_type(pid):
    try:
        int(pid)
        return 'int'
    except:
        return 'str'

impressions_df['pid'] = impressions_df['placement_ids'].apply(extract_pid)
impressions_df['pid_type'] = impressions_df['pid'].apply(pid_type)
impressions_df = impressions_df[impressions_df['pid_type'] == 'int']
impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))

In [8]:
omg_placement_ids = list(omg_impressions["Placement Id"].unique())
impressions_df = impressions_df[impressions_df["pid"].isin(omg_placement_ids)]

In [9]:
def _get_final_fixation(gaze_valid, is_fixated, id_, pred_map):
    if gaze_valid:
        return is_fixated
    if id_ in pred_map:
        return pred_map[id_]
    return False

impressions_df['final_fixation'] = impressions_df.apply(lambda row: _get_final_fixation(row['gaze_valid'], row['is_fixated'], row['id'], pred_map), axis=1)

In [10]:
impressions_df["true_is_iab_inview"] = impressions_df.apply(
    lambda x: 
    True if (
        (x["ad_technical_format"] == "out-stream") & (x["exist_viewable_1_s_threshold_50"] == True) & (x["exist_viewable_2_s_threshold_50"] == False)
    ) | (x["is_iab_inview"] == True) 
    else False, 
    axis=1
)

In [11]:
grouped_df = impressions_df.groupby(["pid", "hostname"]).apply(
    lambda x: pd.Series(
        {
            "num_impressions": x["id"].count(),
            "num_fixations": x.loc[x["final_fixation"], "id"].count(),
            "is_iab_inview": x.loc[x["true_is_iab_inview"], "id"].count(),
        }
    )
)
grouped_df = grouped_df.reset_index()

In [12]:
grouped_df["fixation_ratio"] = round(grouped_df["num_fixations"] / grouped_df["num_impressions"] * 100, 2)
grouped_df["inview_ratio"] = round(grouped_df["is_iab_inview"] / grouped_df["num_impressions"] * 100, 2)
grouped_df

,pid,hostname,num_impressions,num_fixations,is_iab_inview,fixation_ratio,inview_ratio
0,14746552,bytbil.com,686,178,373,25.95,54.37
1,14746553,bytbil.com,402,81,197,20.15,49.00
2,19428808,hitta.se,9,0,4,0.00,44.44
3,19461224,klart.se,166,12,31,7.23,18.67
4,19499407,aftonbladet.se,26317,2398,16686,9.11,63.40
...,...,...,...,...,...,...,...
105,22875555,lt.se,2,0,2,0.00,100.00
106,22875580,na.se,2,1,1,50.00,50.00
107,22875659,st.nu,8,0,4,0.00,50.00
108,22875683,vlt.se,1,0,0,0.00,0.00


In [13]:
omg_grouped = omg_impressions.groupby("Placement Id").apply(
    lambda x: pd.Series(
        {
            "client_impressions" : x["Imps"].sum()
        }
    )
)
omg_grouped = omg_grouped.reset_index()

In [14]:
merged_df = pd.merge(grouped_df, omg_grouped, left_on=["pid"], right_on=["Placement Id"], how="outer")
merged_df

,pid,hostname,num_impressions,num_fixations,is_iab_inview,fixation_ratio,inview_ratio,Placement Id,client_impressions
0,14746552.0,bytbil.com,686.0,178.0,373.0,25.95,54.37,14746552,297
1,14746553.0,bytbil.com,402.0,81.0,197.0,20.15,49.00,14746553,221
2,19428808.0,hitta.se,9.0,0.0,4.0,0.00,44.44,19428808,547
3,19461224.0,klart.se,166.0,12.0,31.0,7.23,18.67,19461224,38
4,19499407.0,aftonbladet.se,26317.0,2398.0,16686.0,9.11,63.40,19499407,277444
...,...,...,...,...,...,...,...,...,...
270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22875600,3018
271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22875602,162
272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22875625,8559
273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22875629,435


In [15]:
merged_df = merged_df.dropna(subset=["pid"])
merged_df

,pid,hostname,num_impressions,num_fixations,is_iab_inview,fixation_ratio,inview_ratio,Placement Id,client_impressions
0,14746552.0,bytbil.com,686.0,178.0,373.0,25.95,54.37,14746552,297
1,14746553.0,bytbil.com,402.0,81.0,197.0,20.15,49.00,14746553,221
2,19428808.0,hitta.se,9.0,0.0,4.0,0.00,44.44,19428808,547
3,19461224.0,klart.se,166.0,12.0,31.0,7.23,18.67,19461224,38
4,19499407.0,aftonbladet.se,26317.0,2398.0,16686.0,9.11,63.40,19499407,277444
...,...,...,...,...,...,...,...,...,...
105,22875555.0,lt.se,2.0,0.0,2.0,0.00,100.00,22875555,5751
106,22875580.0,na.se,2.0,1.0,1.0,50.00,50.00,22875580,14140
107,22875659.0,st.nu,8.0,0.0,4.0,0.00,50.00,22875659,3728
108,22875683.0,vlt.se,1.0,0.0,0.0,0.00,0.00,22875683,7995


In [16]:
merged_df["is_iab_inview_client_based_imps"] = merged_df.apply(lambda x: x["client_impressions"] * (x["inview_ratio"] / 100), axis=1)
merged_df["client_based_fixations"] = merged_df.apply(lambda x: x["client_impressions"] * (x["fixation_ratio"] / 100), axis=1)
merged_df["client_fixation_rate"] = round(merged_df["client_based_fixations"] / merged_df["client_impressions"] * 100, 2)
merged_df

/tmp/ipykernel_34625/3224130395.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df["is_iab_inview_client_based_imps"] = merged_df.apply(lambda x: x["client_impressions"] * (x["inview_ratio"] / 100), axis=1)
/tmp/ipykernel_34625/3224130395.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df["client_based_fixations"] = merged_df.apply(lambda x: x["client_impressions"] * (x["fixation_ratio"] / 100), axis=1)
/tmp/ipykernel_34625/3224130395.py:3: SettingWithCopyWarning: 
A value is trying to

,pid,hostname,num_impressions,num_fixations,is_iab_inview,fixation_ratio,inview_ratio,Placement Id,client_impressions,is_iab_inview_client_based_imps,client_based_fixations,client_fixation_rate
0,14746552.0,bytbil.com,686.0,178.0,373.0,25.95,54.37,14746552,297,161.4789,77.0715,25.95
1,14746553.0,bytbil.com,402.0,81.0,197.0,20.15,49.00,14746553,221,108.2900,44.5315,20.15
2,19428808.0,hitta.se,9.0,0.0,4.0,0.00,44.44,19428808,547,243.0868,0.0000,0.00
3,19461224.0,klart.se,166.0,12.0,31.0,7.23,18.67,19461224,38,7.0946,2.7474,7.23
4,19499407.0,aftonbladet.se,26317.0,2398.0,16686.0,9.11,63.40,19499407,277444,175899.4960,25275.1484,9.11
...,...,...,...,...,...,...,...,...,...,...,...,...
105,22875555.0,lt.se,2.0,0.0,2.0,0.00,100.00,22875555,5751,5751.0000,0.0000,0.00
106,22875580.0,na.se,2.0,1.0,1.0,50.00,50.00,22875580,14140,7070.0000,7070.0000,50.00
107,22875659.0,st.nu,8.0,0.0,4.0,0.00,50.00,22875659,3728,1864.0000,0.0000,0.00
108,22875683.0,vlt.se,1.0,0.0,0.0,0.00,0.00,22875683,7995,0.0000,0.0000,0.00


In [17]:
merged_df["num_fixations"] = merged_df["num_fixations"].astype("Int64")
merged_df["num_impressions"] = merged_df["num_impressions"].astype("Int64")

/tmp/ipykernel_34625/94787776.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df["num_fixations"] = merged_df["num_fixations"].astype("Int64")
/tmp/ipykernel_34625/94787776.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df["num_impressions"] = merged_df["num_impressions"].astype("Int64")


In [18]:
def get_sample_size(num_impressions):
    if np.isnan(num_impressions):
        return "N/A"
    if num_impressions <= 20:
        return "low"
    elif num_impressions > 20 and num_impressions <= 100:
        return "medium"
    else:
        return "high"
    
merged_df['sample_size'] = merged_df['num_impressions'].apply(lambda num_impressions: get_sample_size(num_impressions))

/tmp/ipykernel_34625/2760417538.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['sample_size'] = merged_df['num_impressions'].apply(lambda num_impressions: get_sample_size(num_impressions))


In [20]:
result_df = merged_df[["hostname", "Placement Id", "client_impressions", "client_based_fixations", "is_iab_inview_client_based_imps", "num_impressions"]]
result_df["fixation_ratio"] = round(result_df["client_based_fixations"] / result_df["client_impressions"] * 100, 2)
result_df["inview_ratio"] = round(result_df["is_iab_inview_client_based_imps"] / result_df["client_impressions"] * 100, 2)
result_df["is_iab_inview_client_based_imps"] = np.floor(result_df["is_iab_inview_client_based_imps"]).astype("Int64")
result_df["client_based_fixations"] = np.floor(result_df["client_based_fixations"]).astype("Int64")
result_df

/tmp/ipykernel_34625/1849696933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["fixation_ratio"] = round(result_df["client_based_fixations"] / result_df["client_impressions"] * 100, 2)
/tmp/ipykernel_34625/1849696933.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["inview_ratio"] = round(result_df["is_iab_inview_client_based_imps"] / result_df["client_impressions"] * 100, 2)
/tmp/ipykernel_34625/1849696933.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

,hostname,Placement Id,client_impressions,client_based_fixations,is_iab_inview_client_based_imps,num_impressions,fixation_ratio,inview_ratio
0,bytbil.com,14746552,297,77,161,686,25.95,54.37
1,bytbil.com,14746553,221,44,108,402,20.15,49.00
2,hitta.se,19428808,547,0,243,9,0.00,44.44
3,klart.se,19461224,38,2,7,166,7.23,18.67
4,aftonbladet.se,19499407,277444,25275,175899,26317,9.11,63.40
...,...,...,...,...,...,...,...,...
105,lt.se,22875555,5751,0,5751,2,0.00,100.00
106,na.se,22875580,14140,7070,7070,2,50.00,50.00
107,st.nu,22875659,3728,0,1864,8,0.00,50.00
108,vlt.se,22875683,7995,0,0,1,0.00,0.00


In [25]:
final_grouped_df = result_df.groupby(["hostname"]).apply(
    lambda x: pd.Series(
        {
            "Impressions": x["client_impressions"].sum(),
            "Fixations": x["client_based_fixations"].sum(),
            "Fixation Ratio": round(x["client_based_fixations"].sum() / x["client_impressions"].sum() * 100, 2),
            "Inviews": x["is_iab_inview_client_based_imps"].sum(),
            "Inview Ratio": round(x["is_iab_inview_client_based_imps"].sum() / x["client_impressions"].sum() * 100, 2),
            "num_impressions": x["num_impressions"].sum(),
        }
    )
)
final_df = final_grouped_df.reset_index()
final_df

,hostname,Impressions,Fixations,Fixation Ratio,Inviews,Inview Ratio,num_impressions
0,aftonbladet.se,1378770.0,362838.0,26.32,746514.0,54.14,206498.0
1,alltommat.expressen.se,1049.0,0.0,0.00,524.0,49.95,2.0
2,blocket.se,6793.0,467.0,6.87,3631.0,53.45,13992.0
3,bytbil.com,518.0,121.0,23.36,269.0,51.93,1088.0
4,classic.prisjakt.nu,142.0,0.0,0.00,0.0,0.00,507.0
5,damernasvarld.expressen.se,9713.0,0.0,0.00,9713.0,100.00,2.0
6,dn.se,620540.0,41061.0,6.62,450479.0,72.59,1465.0
7,expressen.se,1008044.0,150833.0,14.96,638991.0,63.39,6065.0
8,hd.se,39177.0,0.0,0.00,1591.0,4.06,7.0
9,hemnet.se,56.0,0.0,0.00,10.0,17.86,11.0


In [26]:
final_df['sample_size'] = final_df['num_impressions'].apply(lambda num_impressions: get_sample_size(num_impressions))

In [27]:
final_df = final_df[["hostname", "Impressions", "Fixations", "Fixation Ratio", "Inviews", "Inview Ratio", "sample_size"]]
final_df = final_df.rename(columns={"sample_size": "Sample Size"})

In [30]:
final_df

,hostname,Impressions,Fixations,Fixation Ratio,Inviews,Inview Ratio,Sample Size
0,aftonbladet.se,1378770.0,362838.0,26.32,746514.0,54.14,high
1,alltommat.expressen.se,1049.0,0.0,0.00,524.0,49.95,low
2,blocket.se,6793.0,467.0,6.87,3631.0,53.45,high
3,bytbil.com,518.0,121.0,23.36,269.0,51.93,high
4,classic.prisjakt.nu,142.0,0.0,0.00,0.0,0.00,high
5,damernasvarld.expressen.se,9713.0,0.0,0.00,9713.0,100.00,low
6,dn.se,620540.0,41061.0,6.62,450479.0,72.59,high
7,expressen.se,1008044.0,150833.0,14.96,638991.0,63.39,high
8,hd.se,39177.0,0.0,0.00,1591.0,4.06,low
9,hemnet.se,56.0,0.0,0.00,10.0,17.86,low


In [33]:
final_df["Inviews"] = final_df["Inviews"].astype("Int64")
final_df["Impressions"] = final_df["Impressions"].astype("Int64")
final_df["Fixations"] = final_df["Fixations"].astype("Int64")

In [34]:
final_df

,hostname,Impressions,Fixations,Fixation Ratio,Inviews,Inview Ratio,Sample Size
0,aftonbladet.se,1378770,362838,26.32,746514,54.14,high
1,alltommat.expressen.se,1049,0,0.00,524,49.95,low
2,blocket.se,6793,467,6.87,3631,53.45,high
3,bytbil.com,518,121,23.36,269,51.93,high
4,classic.prisjakt.nu,142,0,0.00,0,0.00,high
5,damernasvarld.expressen.se,9713,0,0.00,9713,100.00,low
6,dn.se,620540,41061,6.62,450479,72.59,high
7,expressen.se,1008044,150833,14.96,638991,63.39,high
8,hd.se,39177,0,0.00,1591,4.06,low
9,hemnet.se,56,0,0.00,10,17.86,low


In [35]:
final_df.to_excel("../final_reports/OMD_xandr_enrichment_v4.xlsx", index=False)